In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import cv2
import random
import os

In [5]:
def seed_everything(seed):
    random.seed(seed)
#     os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(2019)

In [80]:
# create meta using csv only need run once
train_df = pd.read_csv('trainLabels15.csv')
test_df = pd.read_csv('testLabels15.csv')

train_meta = pd.DataFrame()
for i in range(len(train_df)):
    df_i = train_df.iloc[i]
    image_name = df_i['image_name']
    image_path = os.path.join(df_i['base_dir'], df_i['source'], image_name)
    image = cv2.imread(image_path)
    h, w, _ = image.shape
    t_meta = pd.DataFrame([{'image': image_name.split('.')[0], 'h': h, 'w': w}])
    train_meta = pd.concat([train_meta, t_meta])
    
# train_meta.head()
# train_meta.to_csv('train_meta.csv', index=False)

test_meta = pd.DataFrame()
for i in range(len(test_df)):
    df_i = test_df.iloc[i]
    if df_i['Usage'] == 'Public':
        continue
    image_name = df_i['image_name']
    image_path = os.path.join(df_i['base_dir'], df_i['source'], image_name)
    image = cv2.imread(image_path)
    h, w, _ = image.shape
    t_meta = pd.DataFrame([{'image': image_name, 'h': h, 'w': w}])
    test_meta = pd.concat([test_meta, t_meta])
test_meta.head()
test_meta.to_csv('test_meta.csv', index=False)

In [81]:
train_meta = pd.read_csv('train_meta.csv')
# train_meta = train_meta[['image', 'h', 'w']]
# train_meta['image'] = train_meta['image'].apply(lambda x: x.split('.')[0])
# train_meta.to_csv('train_meta.csv', index=False)
test_meta = pd.read_csv('test_meta.csv')
test_meta = test_meta[['image', 'h', 'w']]
test_meta['image'] = test_meta['image'].apply(lambda x: x.split('.')[0])
test_meta.to_csv('test_meta.csv', index=False)

In [82]:
train_post = pd.read_csv('train_post.csv')
test_post = pd.read_csv('test_post.csv')
test_post = test_post[['image', 'pre0',	'pre1',	'pre2',	'pre3',	'pre4']]

In [83]:
train_meta

,image,h,w
0,10_left,1019,1024
1,10_right,1019,1023
2,13_left,981,1024
3,13_right,981,1024
4,15_left,1023,1024
...,...,...,...
35121,44347_right,1022,1024
35122,44348_left,851,1024
35123,44348_right,851,1024
35124,44349_left,800,1024


In [20]:
train_post

,image,pre0,pre1,pre2,pre3,pre4,level
0,10_left,0.603337,0.181160,0.175605,0.020245,0.019654,0
1,10_right,0.642514,0.043781,0.236506,0.037295,0.039904,0
2,13_left,0.881163,0.072912,0.016002,0.013922,0.016001,0
3,13_right,0.903917,0.043543,0.015409,0.016503,0.020628,0
4,15_left,0.791390,0.106171,0.050869,0.028083,0.023487,1
...,...,...,...,...,...,...,...
35121,44347_right,0.912321,0.043721,0.014218,0.014148,0.015592,0
35122,44348_left,0.869755,0.063954,0.024336,0.017748,0.024208,0
35123,44348_right,0.817282,0.120720,0.025326,0.016390,0.020281,0
35124,44349_left,0.873581,0.060041,0.026183,0.020552,0.019643,0


In [84]:
train_post2 = pd.merge(train_post, train_meta)

In [85]:
train_labels = train_post2['level']

In [86]:
train_post2 = train_post2.drop(['level'], axis=1)

In [87]:
test_post2 = pd.merge(test_post, test_meta)

In [91]:
test_post2.head(1)

,image,pre0,pre1,pre2,pre3,pre4,h,w
0,1_left,0.865235,0.023446,0.077598,0.019994,0.013728,807,1024


### left和right合并

In [92]:
train_post2_values = train_post2.values
test_post2_values = test_post2.values

In [93]:
train_dic = {}
for i in range(len(train_post2_values)):
    line_list = train_post2_values[i].tolist()
    train_dic[line_list[0]] = line_list[1:]
#     break

In [14]:
# train_dic

In [94]:
train_list = []
for i in range(len(train_post2_values)):
    img = train_post2_values[i][0]
    img_id, lr = img.split('_')
    if lr == 'left':
        train_list.append(train_dic[img] + train_dic[img_id+'_right'])
    if lr == 'right':
        train_list.append(train_dic[img] + train_dic[img_id+'_left'])
#     break

In [95]:
train_list[0]

[0.6033366322517395,
 0.1811595112085342,
 0.1756049990653991,
 0.020245037972927,
 0.0196538232266902,
 1019,
 1024,
 0.6425143480300903,
 0.0437805205583572,
 0.2365061342716217,
 0.0372953787446022,
 0.0399036966264247,
 1019,
 1023]

In [96]:
test_dic = {}
for i in range(len(test_post2_values)):
    line_list = test_post2_values[i].tolist()
    test_dic[line_list[0]] = line_list[1:]
#     break
test_list = []
for i in range(len(test_post2_values)):
    img = test_post2_values[i][0]
    img_id, lr = img.split('_')
    if lr == 'left':
        test_list.append(test_dic[img] + test_dic[img_id+'_right'])
    if lr == 'right':
        test_list.append(test_dic[img] + test_dic[img_id+'_left'])
#     break

In [97]:
test_list[0]

[0.8652345538139343,
 0.023445900529623,
 0.0775982439517974,
 0.019993633031845,
 0.013727780431509,
 807,
 1024,
 0.8980518579483032,
 0.0499805472791194,
 0.0237993020564317,
 0.0137928314507007,
 0.0143754556775093,
 805,
 1024]

### xgb

In [98]:
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold

xgb_params = {
#     'max_delta_step': 0,
    'max_depth': 5,
    'eval_metric': 'rmse',
    'seed': 111,
#     'eta': 0.0123,
#     'subsample': 0.8,
#     'colsample_bytree': 0.85,
    # 'tree_method': 'gpu_hist',
    # 'device': 'gpu',
#     'silent': 1,
}

In [99]:
def run_xgb(params, X_train, y, X_test):
    n_splits = 8
    verbose_eval = 50
    num_rounds = 30000
    early_stop = 50

    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=111)

    oof_train = np.zeros((len(X_train)))
    oof_test = np.zeros((len(X_test), n_splits))

    i = 0

    for train_idx, valid_idx in kf.split(X_train, y):
#         print(train_idx)
        X_tr = X_train[train_idx]
        X_val = X_train[valid_idx]

        y_tr = y[train_idx]
        y_val = y[valid_idx]

        d_train = xgb.DMatrix(data=X_tr, label=y_tr)
        d_valid = xgb.DMatrix(data=X_val, label=y_val)

        watchlist = [(d_train, 'train'), (d_valid, 'valid')]
        model = xgb.train(dtrain=d_train, num_boost_round=num_rounds, evals=watchlist,
                         early_stopping_rounds=early_stop, verbose_eval=verbose_eval, params=params)

        valid_pred = model.predict(xgb.DMatrix(X_val), ntree_limit=model.best_ntree_limit)
        test_pred = model.predict(xgb.DMatrix(X_test), ntree_limit=model.best_ntree_limit)

        oof_train[valid_idx] = valid_pred
        oof_test[:, i] = test_pred

        i += 1
    return model, oof_train, oof_test

In [100]:
# print(np.array(train_list).shape)
# print(np.array(train_labels).shape)
# print(np.array(test_list).shape)
model, oof_train, oof_test = run_xgb(xgb_params, np.array(train_list), np.array(train_labels), np.array(test_list))

[0]	train-rmse:0.73414	valid-rmse:0.73812
[50]	train-rmse:0.33253	valid-rmse:0.39443
[67]	train-rmse:0.32179	valid-rmse:0.39484
[0]	train-rmse:0.73401	valid-rmse:0.73646


/home/pudding/Software/anaconda3/envs/cls_dr/lib/python3.8/site-packages/xgboost/core.py:105: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


[50]	train-rmse:0.33339	valid-rmse:0.38532
[66]	train-rmse:0.32543	valid-rmse:0.38677
[0]	train-rmse:0.73396	valid-rmse:0.73537
[50]	train-rmse:0.33303	valid-rmse:0.39487
[61]	train-rmse:0.32738	valid-rmse:0.39540
[0]	train-rmse:0.73374	valid-rmse:0.73767
[50]	train-rmse:0.33050	valid-rmse:0.39842
[72]	train-rmse:0.31782	valid-rmse:0.40114
[0]	train-rmse:0.73436	valid-rmse:0.73643
[50]	train-rmse:0.33513	valid-rmse:0.38848
[59]	train-rmse:0.32730	valid-rmse:0.39091
[0]	train-rmse:0.73446	valid-rmse:0.73560
[50]	train-rmse:0.33692	valid-rmse:0.39283
[63]	train-rmse:0.32838	valid-rmse:0.39430
[0]	train-rmse:0.73421	valid-rmse:0.73752
[50]	train-rmse:0.33327	valid-rmse:0.39279
[77]	train-rmse:0.31632	valid-rmse:0.39620
[0]	train-rmse:0.73350	valid-rmse:0.73684
[50]	train-rmse:0.33209	valid-rmse:0.41252
[58]	train-rmse:0.32640	valid-rmse:0.41415


In [101]:
from sklearn.metrics import cohen_kappa_score, mean_squared_error
from sklearn.metrics import confusion_matrix as sk_cmatrix


# FROM: https://www.kaggle.com/myltykritik/simple-lgbm-image-features

# The following 3 functions have been taken from Ben Hamner's github repository
# https://github.com/benhamner/Metrics
def confusion_matrix(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    Returns the confusion matrix between rater's ratings
    """
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(rater_a + rater_b)
    if max_rating is None:
        max_rating = max(rater_a + rater_b)
    num_ratings = int(max_rating - min_rating + 1)
    conf_mat = [[0 for i in range(num_ratings)]
                for j in range(num_ratings)]
    for a, b in zip(rater_a, rater_b):
        conf_mat[a - min_rating][b - min_rating] += 1
    return conf_mat


def histogram(ratings, min_rating=None, max_rating=None):
    """
    Returns the counts of each type of rating that a rater made
    """
    if min_rating is None:
        min_rating = min(ratings)
    if max_rating is None:
        max_rating = max(ratings)
    num_ratings = int(max_rating - min_rating + 1)
    hist_ratings = [0 for x in range(num_ratings)]
    for r in ratings:
        hist_ratings[r - min_rating] += 1
    return hist_ratings


def quadratic_weighted_kappa(y, y_pred):
    """
    Calculates the quadratic weighted kappa
    axquadratic_weighted_kappa calculates the quadratic weighted kappa
    value, which is a measure of inter-rater agreement between two raters
    that provide discrete numeric ratings.  Potential values range from -1
    (representing complete disagreement) to 1 (representing complete
    agreement).  A kappa value of 0 is expected if all agreement is due to
    chance.
    quadratic_weighted_kappa(rater_a, rater_b), where rater_a and rater_b
    each correspond to a list of integer ratings.  These lists must have the
    same length.
    The ratings should be integers, and it is assumed that they contain
    the complete range of possible ratings.
    quadratic_weighted_kappa(X, min_rating, max_rating), where min_rating
    is the minimum possible rating, and max_rating is the maximum possible
    rating
    """
    rater_a = y
    rater_b = y_pred
    min_rating=None
    max_rating=None
    rater_a = np.array(rater_a, dtype=int)
    rater_b = np.array(rater_b, dtype=int)
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(min(rater_a), min(rater_b))
    if max_rating is None:
        max_rating = max(max(rater_a), max(rater_b))
    conf_mat = confusion_matrix(rater_a, rater_b,
                                min_rating, max_rating)
    num_ratings = len(conf_mat)
    num_scored_items = float(len(rater_a))

    hist_rater_a = histogram(rater_a, min_rating, max_rating)
    hist_rater_b = histogram(rater_b, min_rating, max_rating)

    numerator = 0.0
    denominator = 0.0

    for i in range(num_ratings):
        for j in range(num_ratings):
            expected_count = (hist_rater_a[i] * hist_rater_b[j]
                              / num_scored_items)
            d = pow(i - j, 2.0) / pow(num_ratings - 1, 2.0)
            numerator += d * conf_mat[i][j] / num_scored_items
            denominator += d * expected_count / num_scored_items

    return (1.0 - numerator / denominator)

In [102]:
test_y = pd.read_csv('test_post.csv').level

import numpy as np
import pandas as pd
import os
import scipy as sp
from functools import partial
from sklearn import metrics
from collections import Counter
import json
from sklearn import metrics


class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        X_p = np.copy(X)
        X_list = []
        for i in X_p:
            X_list.append(float(i))
#         print(X_list)
        X_list = pd.cut(X_list, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3, 4])

        ll = metrics.cohen_kappa_score(y, X_list, weights='quadratic')
        #ll = metrics.accuracy_score(y, X_list)
        return -ll

    def fit(self, X, y):
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        initial_coef = [0.5, 1.5, 2.5, 3.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead')
        print(-loss_partial(self.coef_['x']))

    def predict(self, X, coef):
        X_p = np.copy(X)
        X_list = []
        for i in X_p:
            X_list.append(float(i))
#         print(X_list)
        X_list = pd.cut(X_list, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3, 4])
        return X_list

    def coefficients(self):
        return self.coef_['x']
        
optR = OptimizedRounder()
optR.fit(oof_train,train_labels)
coefficients = optR.coefficients()

0.9040493964566227


In [103]:
coefficients

array([0.51434632, 1.48079988, 2.32702486, 3.47527765])

In [104]:
oof_test.mean(1)

array([0.04052587, 0.02139361, 1.91582611, ..., 0.15683182, 0.02180919,
       0.01716423])

In [105]:
from sklearn.metrics import accuracy_score

test_predictions = optR.predict(oof_test.mean(1), coefficients)

print(test_predictions)

# print('acc:',len(acc) / len(sample_df))

print('qwk:',metrics.cohen_kappa_score(test_y, list(test_predictions), weights='quadratic'))
print('acc:',accuracy_score(list(test_predictions), test_y))
# sample_df.to_csv("../resnet18_all_optacc.csv", index=None)

[0, 0, 2, 1, 1, ..., 1, 0, 0, 0, 0]
Length: 42670
Categories (5, int64): [0 < 1 < 2 < 3 < 4]
qwk: 0.8539648302261059
acc: 0.8425123037262714


qwk: 0.8479403542087205
acc: 0.8373338808421681

In [56]:
from sklearn.metrics import confusion_matrix
confusion_matrix(
    test_y,   # array, Gound true (correct) target values
    list(test_predictions),  # array, Estimated targets as returned by a classifier
)

array([[37621,  1473,   395,    28,    16],
       [ 1846,  1325,   584,     5,     2],
       [  976,  1025,  4880,   905,    75],
       [   23,    17,   364,   750,    60],
       [   20,    24,   139,   384,   639]])

## Test seed

In [30]:
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

for i in [11,111,11111,33]:
    xgb_params = {
    #     'max_delta_step': 0,
        'max_depth': 5,
        'eval_metric': 'rmse',
        'seed': i,
    #     'eta': 0.0123,
    #     'subsample': 0.8,
    #     'colsample_bytree': 0.85,
        'tree_method': 'gpu_hist',
        'device': 'gpu',
        'verbose': False,
    #     'silent': 1,
    }

    model, oof_train, oof_test = run_xgb(xgb_params, np.array(train_list), np.array(train_labels), np.array(test_list),i)

    optR = OptimizedRounder()
    optR.fit(oof_train,train_labels)
    coefficients = optR.coefficients()
    oof_test.mean(1)

    test_predictions = optR.predict(oof_test.mean(1), coefficients)

    test_predictions

    # print('acc:',len(acc) / len(sample_df))

    print('qwk:',metrics.cohen_kappa_score(test_y, list(test_predictions), weights='quadratic'))
    print('acc:',accuracy_score(list(test_predictions), test_y))
    # sample_df.to_csv("../resnet18_all_optacc.csv", index=None)

[20:35:05] WARNING: ../src/learner.cc:516: 
Parameters: { device, verbose } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-rmse:0.74634	valid-rmse:0.74849
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 50 rounds.
[50]	train-rmse:0.37255	valid-rmse:0.43148
Stopping. Best iteration:
[13]	train-rmse:0.40170	valid-rmse:0.42878

[20:35:05] WARNING: ../src/learner.cc:516: 
Parameters: { device, verbose } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-rmse:0.74706	valid-r

KeyboardInterrupt: 

### RF

In [123]:
# from sklearn.model_selection import GridSearchCV #网格搜索
# param_test1 = {'n_estimators':range(50,501,50)}
# gsearch1 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1, verbose=True), 
# param_grid = param_test1,n_jobs=20,iid=False, cv=5, verbose=True)
# gsearch1.fit(train_list,train_labels)

In [124]:
# means = gsearch1.cv_results_['mean_test_score']
# params = gsearch1.cv_results_['params']
# for mean,param in zip(means,params):
#     print("%f  with:   %r" % (mean,param))

In [125]:
from sklearn.ensemble import GradientBoostingRegressor

In [126]:
gbdt = GradientBoostingRegressor(n_estimators=70, verbose=True)

In [127]:
gbdt.fit(train_list,train_labels)

      Iter       Train Loss   Remaining Time 
         1           0.8013           11.30s
         2           0.6871           11.76s
         3           0.5943           11.19s
         4           0.5190           10.89s
         5           0.4563           10.55s
         6           0.4059           10.28s
         7           0.3644           10.03s
         8           0.3299            9.82s
         9           0.3022            9.63s
        10           0.2792            9.53s
        20           0.1856            7.79s
        30           0.1703            6.21s
        40           0.1665            4.66s
        50           0.1646            3.12s
        60           0.1634            1.55s
        70           0.1624            0.00s


GradientBoostingRegressor(n_estimators=70, verbose=True)

In [128]:
preds = gbdt.predict(test_list)

In [129]:
train_preds = gbdt.predict(train_list)

In [130]:
test_y = pd.read_csv('../../d/testLabels15.csv').level

import numpy as np
import pandas as pd
import os
import scipy as sp
from functools import partial
from sklearn import metrics
from collections import Counter
import json
from sklearn import metrics


class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        X_p = np.copy(X)
        X_list = []
        for i in X_p:
            X_list.append(float(i))
#         print(X_list)
        X_list = pd.cut(X_list, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3, 4])

        ll = metrics.cohen_kappa_score(y, X_list, weights='quadratic')
        #ll = metrics.accuracy_score(y, X_list)
        return -ll

    def fit(self, X, y):
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        initial_coef = [0.5, 1.5, 2.5, 3.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead')
        print(-loss_partial(self.coef_['x']))

    def predict(self, X, coef):
        X_p = np.copy(X)
        X_list = []
        for i in X_p:
            X_list.append(float(i))
#         print(X_list)
        X_list = pd.cut(X_list, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3, 4])
        return X_list

    def coefficients(self):
        return self.coef_['x']
        
optR = OptimizedRounder()
optR.fit(train_preds,train_labels)
coefficients = optR.coefficients()

# remove zoom from FastAI TTA
tta_params = {'beta':0.4, 'scale':1.1}
# preds, y = learn.TTA(ds_type=DatasetType.Test, **tta_params)
# preds,y = learn.get_preds(DatasetType.Test)

print(coefficients)

test_predictions = optR.predict(preds, coefficients)

test_predictions

# print('acc:',len(acc) / len(sample_df))

print('qwk:',metrics.cohen_kappa_score(test_y, list(test_predictions), weights='quadratic'))

# sample_df.to_csv("../resnet18_all_optacc.csv", index=None)

0.8977024575430294
[0.57572504 1.31943508 2.25773514 3.16354651]
qwk: 0.853969924890469
